In [16]:
#Function to plot the correct graph
def plot(data,col,gratyp,name,targ=None):
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sb
    if gratyp=='hist':
        gname='hist_'+str(col)
        plt.figure(figsize=(12,10))
        sb.distplot(data[col],kde=False)#Histogram using Seaborn
        plt.ylabel('Count')
        plt.savefig(gname+'.png',format='png',dpi=400) #Saving the graph as png format with a good dpi
        plt.clf()
    elif gratyp=='density':
        gname='density_'+str(col)
        plt.figure(figsize=(12,10))
        sb.distplot(data[col],hist=False) #Density plot using Seaborn
        plt.ylabel('Density')
        plt.savefig(gname+'.png',format='png',dpi=400) #Saving the graph as png format with a good dpi
        plt.clf()
    elif gratyp=='scatter':
        plt.figure(figsize=(12,10))
        sb.lmplot(x=name,y=targ,data=data,fit_reg=False) #Scatter plot using Seaborn
        gname='scatter_'+str(col)+'_vs_'+targ #Naming the graph
        plt.savefig(gname+'.png',format='png',dpi=400) #Saving the graph as png format with a good dpi
        plt.clf()
    elif gratyp=='bar':
        plt.figure(figsize=(12,10))    
        plt.xticks(rotation='vertical')
        sb.countplot(col,data=data) #Bar graphs using Seaborn
        gname='bar_'+str(col) #Naming the graph
        plt.savefig(gname+'.png',format='png',dpi=400)
        plt.clf()
    elif gratyp=='crosst':
        plt.figure(figsize=(12,10))
        sb.countplot(x=str(col),hue=targ,data=data) #Cross tab count graphs using seaborn
        gname='crosstab_'+str(col)+'_vs_'+targ #Naming the graph
        plt.savefig(gname+'.png',format='png',dpi=400) #Saving the graph as png format with a good dpi
        plt.clf()
    elif gratyp=='box':
        plt.figure(figsize=(12,10))
        sb.boxplot(data[col])
        gname='box_'+str(col) #Naming the graph
        plt.savefig(gname+'.png',format='png',dpi=400) #Saving the graph as png format with a good dpi
        plt.clf()

In [9]:
#Function to check the variable type of a column
def vartype(data,col,direc):
    import pandas as pd
    import os
    os.chdir(direc) #Changing working directory to the required one
    file=pd.read_csv(data)
    try:
        file.loc[0,col] #Getting the column while is currently being iterated in graph function
        #Deciding the proportion of unique values required in a column
        if file.shape[0]<100:
            prop=0.2
        elif file.shape[0]<200:
            prop=0.1
        elif file.shape[0]<1000:
            prop=0.05
        else:
            prop=0.03
        if ((file[col].dtype=='float64' or file[col].dtype=='int64') and len(file[col].unique())>(prop*file.shape[0])):
            vtype='num'
            return(vtype)
        else:
            vtype='cat' 
            return(vtype)
    except:
        vtype='noexist'
        return(vtype)
                    

In [8]:
#Function to handle folder for saving graphs
def direcheck(direc,fname,graph):
    import os
    if not os.path.exists(direc+'\\'+fname[0:fname.find('.csv')]):#Checking if main folder exists
        os.mkdir(fname[0:fname.find('.csv')]) #Making main directory if it doesnt exist
        dire=direc+'\\'+fname[0:fname.find('.csv')] 
        os.chdir(dire) #Changing working directory to the main directory
        os.mkdir(graph) #Creating sub directory indicating the type of graph
        dire=direc+'\\'+fname[0:fname.find('.csv')]+'\\'+graph 
        os.chdir(dire) #Changing working directory to sub directory
    else:
        dire=direc+'\\'+fname[0:fname.find('.csv')] 
        os.chdir(dire)
        if not os.path.exists(direc+'\\'+fname[0:fname.find('.csv')]+'\\'+graph): #If main directory exists we check for required sub directory
            os.mkdir(graph) #If that doesnt exist a new sub directory is created
            dire=direc+'\\'+fname[0:fname.find('.csv')]+'\\'+graph 
            os.chdir(dire) #Changing working directory to sub directory
        else:
            dire=direc+'\\'+fname[0:fname.find('.csv')]+'\\'+graph
            os.chdir(dire) #If both main and sub directory exist we just change working directory to sub directory

In [18]:
#Main graph function which we will call
# Features- a) Takes target variable as argument to print scatter plots (if numerical) 
# b) Can apply log and sqrt transformation through optional argument
# c) Take imputation as an optional argument to impute missing values by mean or mode
# d) Folder management
def graph(fname,col=None,direc='D:\\DS\\Machine learning\\Datasets',targ=None,transf=None,impu=None): #Set the direc path to your default dataset directory
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sb
    import numpy as np
    plt.rc('figure', max_open_warning = 0)
    try:
        os.chdir(direc) #Set working directory
    except:
        print('Path doesnt exist.Exiting')
        return
    try:
        read=pd.read_csv(fname) #Reading the dataset
    except:
        print('File doesnt exist')
        return
    if col==None:
        colu=read.columns #Setting list of columns to be used for plotting
    else:
        if col.find(',')==-1:
            colu=list([col]) #Setting list of columns to be used for plotting
        else:
            colu=col #Setting list of columns to be used for plotting
    targtype=vartype(fname,targ,direc) #Checking variable type of target variable
    if targtype=='noexist' and targ!=None:   
        print('Target column doesnt exist')
    if impu=='yes' or impu=='Yes':
        for i in colu:
            vtype=vartype(fname,i,direc)
            if vtype=='num':
                if read[i].isnull().sum()>0:
                    read.loc[read[str(i)].isnull(),str(i)]=read[str(i)].mean() #Mean imputation
            elif vtype=='cat':
                if read[i].isnull().sum()>0:
                    read.loc[read[str(i)].isnull(),str(i)]=read[str(i)].mode() #Mode imputation
    for i in colu: #Iterating over the columns
        vtype=vartype(fname,i,direc) #Checking variable type of currently iterating column
        name=str(i)
        if vtype=='num':
            direcheck(direc,fname,'Histogram Graphs') #Checking directories
            plot(read,i,'hist',name)
            direcheck(direc,fname,'Density Plots') #Checking directories
            plot(read,i,'density',name)
            direcheck(direc,fname,'Box Plots') #Checking directories
            plot(read,i,'box',name)
            if targ!=None and targ!=name and targtype=='num':
                direcheck(direc,fname,'Scatter Plots') #Checking directories
                plot(read,i,'scatter',name,targ) #Call plot function
            if transf!=None and targ!=None and targ!=name and targtype=='num':
                if transf=='log':
                    read['temp']=np.log(read[name])# Log transformation
                    direcheck(direc,fname,'Log Transformed Scatter Plots') #Checking directories
                    plot(read,i,'scatter','temp',targ)
                if transf=='sqrt':
                    read['temp']=np.sqrt(read[name]) # Sqrt transformation
                    direcheck(direc,fname,'Sqrt Transformed Scatter Plots') #Checking directories
                    plot(read,i,'scatter','temp',targ) #Calling plot function
        elif vtype=='cat':
            direcheck(direc,fname,'Bar Graphs') #Checking directories
            plot(read,i,'bar',name)
            if targ!=None and targ!=name and targtype=='cat':
                direcheck(direc,fname,'CrossTab Graphs') #Checking directories
                plot(read,i,'crosst',name)
        else:
            print('Column %s'%name+' doesnt exist')

In [23]:
graph('cars.csv',targ='MPG',transf='log')

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in log


<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

...

<Figure size 360x360 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 360x360 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>